# Audio Embedding and Tagging

## Import Packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" 
import json
import torch
import librosa
import openl3
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

E0000 00:00:1745174555.903219  804636 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745174555.909119  804636 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745174555.923913  804636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745174555.923939  804636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745174555.923941  804636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745174555.923942  804636 computation_placer.cc:177] computation placer already registered. Please check linka

## Get Embeddings and Tags

In [ ]:
AUDIO_DIR = "../data/song_audio_files"
OUTPUT_CSV = "openl3_embeddings.csv"
NUM_WORKERS = 10

model = openl3.models.load_audio_embedding_model(
    input_repr="mel256",
    content_type="music",
    embedding_size=512
)

def embed_file(file):
    try:
        if not file.endswith(".mp3"):
            return None
        path = os.path.join(AUDIO_DIR, file)

        # Get total duration
        duration = librosa.get_duration(path=path)
        offset = max(0, duration / 2 - 15)  # Center 30s window

        # Load from middle
        audio, sr = librosa.load(path, sr=48000, mono=True, offset=offset, duration=30)

        # Get embedding
        emb, _ = openl3.get_audio_embedding(audio, sr, model=model)
        avg_emb = emb.mean(axis=0)

        return {"file": file, **{f"dim_{i}": float(val) for i, val in enumerate(avg_emb)}}

    except Exception as e:
        print(f"{file} failed: {e}")
        return None

# === Parallel loop ===
files = os.listdir(AUDIO_DIR)
results = []

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = {executor.submit(embed_file, f): f for f in files}
    for future in as_completed(futures):
        result = future.result()
        if result:
            results.append(result)

df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)
print(f"\nDone. {len(df)} files embedded and saved to {OUTPUT_CSV}")

/tmp/ipykernel_804636/2068293618.py:18: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=path)


 1/10 ━━━━━━━━━━━━━━━━━━━━ 1:32 10s/stepWARNING:tensorflow:5 out of the last 5 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f28e03914e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
 1/10 ━━━━━━━━━━━━━━━━━━━━ 1:33 10s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f28e03914e0> triggered tf.function retracing. Tracing is

/tmp/ipykernel_804636/2068293618.py:18: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=path)


10/10 ━━━━━━━━━━━━━━━━━━━━ 89s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 90s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 90s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 91s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 91s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 91s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 92s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 92s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 93s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 88s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 89s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 88s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 89s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 88s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 89s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 83s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 83s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━

/home/jah9kqn/.local/lib/python3.11/site-packages/openl3/core.py:55: OpenL3Warning: Provided audio is all zeros
  warnings.warn('Provided audio is all zeros', OpenL3Warning)


10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 83s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 83s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 80s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 83s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 89s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 9s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 8s/step
10/10 ━━━━━━━━━━━━━━━━━━━